This script generates nwchem input file from smiles, selfies and xyz files in the cuurent folder

Depedencies
make_redox_input_script_example.py
derviatives.py

In [ ]:
import numpy as np
from make_redox_input_script_example import RedoxPotentialScript
from rdkit import Chem
from rdkit.Chem import AllChem
import selfies as sf
from openbabel import pybel


In [ ]:

def get_charge_and_multiplicity(mol):
    """Return formal charge and multiplicity from an RDKit molecule."""
    charge = Chem.GetFormalCharge(mol)
    unpaired_electrons = sum(atom.GetNumRadicalElectrons() for atom in mol.GetAtoms())
    multiplicity = unpaired_electrons + 1
    return charge, multiplicity

def smiles_to_xyz(smiles):
    """Convert SMILES to 3D XYZ string using RDKit."""
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # add hydrogens
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # 3D embedding
    AllChem.UFFOptimizeMolecule(mol)             # geometry optimization

    xyz = Chem.MolToXYZBlock(mol)
    return xyz


def get_filetype(filename):
    _, ext = os.path.splitext(filename)
    ext = ext.lower()
    if ext in [".smi", ".smiles"]:
        return "smiles"
    elif ext in [".selfies"]:
        return "selfies"
    elif ext in [".xyz"]:
        return "xyz"
    else:
        raise ValueError(f"Unsupported file extension: {ext}")
    
def input_to_smiles(input_str, filetype="smiles"):
    """Convert SMILES/SELFIES/XYZ input to SMILES string."""
    if filetype == "smiles":
        return input_str.strip()
    elif filetype == "selfies":
        return sf.decoder(input_str.strip())
    elif filetype == "xyz":
        # Use OpenBabel to perceive bonds
        mol = next(pybel.readfile("xyz", input_str))
        mol.addh()
        smiles = mol.write("smi").strip()
        return smiles
    else:
        raise ValueError("filetype must be 'smiles', 'selfies', or 'xyz'")

def analyze_molecule(input_data, filetype="smiles"):
    """Main wrapper: input -> SMILES -> charge, multiplicity, xyz."""
    smiles = input_to_smiles(input_data, filetype)
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Could not parse molecule")

    charge, multiplicity = get_charge_and_multiplicity(mol)
    xyz = smiles_to_xyz(smiles)

    return charge, multiplicity, xyz



In [ ]:

# ---------------------- #
# Example usage:
# ---------------------- #

# # 1) From SMILES
# c, m, xyz = analyze_molecule("CCO", filetype="smiles")
# print("SMILES input: Charge =", c, "Multiplicity =", m)
# print(xyz)

# # 2) From SELFIES
# selfie = "[C][C][O]"  # ethanol
# c, m, xyz = analyze_molecule(selfie, filetype="selfies")
# print("SELFIES input: Charge =", c, "Multiplicity =", m)
# print(xyz)

# 3) From XYZ file
# Save an ethanol.xyz file beforehand
# c, m, xyz = analyze_molecule("ethanol.xyz", filetype="xyz")
# print("XYZ input: Charge =", c, "Multiplicity =", m)
# print(xyz)


In [ ]:
import os

folder_path = "."  # replace with your folder path

for filename in os.listdir(folder_path):
    c, m, name = analyze_molecule(input_data = filename, filetype=get_filetype(filename))
    xyz_path = os.path.join(folder_path, name)
    nwchem_input = RedoxPotentialScript(
        titleGeomOptimizer=f"{name[:-4]}_redox",
        # memory="EMPTY",
        # start = "EMPTY",
        scratch_dir = ".",
        permanent_dir = ".",
        charge = c,
        geometry = name,
        basis = "6-311g*",
        maxIter=300,
        xyz = f"{name[:-4]}_opt",
        xc= "b3lyp",
        mult = m,
        grid  = "medium", # [(xcoarse||coarse||medium||fine||xfine||huge) default medium]
        disp = "vdw 3", # To use the Grimme DFT-D3 dispersion correction, use the option "vdw 3" or "vdw 4"
        taskGeomOptimize = "dft optimize",
        titleFreq = f"{name[:-4]}_FREQ",
        taskFreq = "dft freq",
        titleSolv = f"{name[:-4]}_SOLV",
        dielec = 78.4,
        minbem = "3", # read the doucmentation, not certain what this does
        ificos = "1", # read the doucmentation, not certain what this does
        do_gasphase = "False",
        taskSolvEnergy = "dft energy",
        inputScriptFolderTemplate = None,
    )
    nwchem_input.write_input()